<a href="https://colab.research.google.com/github/nahuelalmeira/poblacionPenal/blob/master/AnalisisYCuracion/DiploDatos_PoblacionPenitenciariaArgentina_AyC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiploDatos 2019 -  Población Penitenciaria en Argentina




> http://diplodatos.famaf.unc.edu.ar/poblacion-penitenciaria-en-argentina-2002-a-2017/




## Practico Análisis y Curación de Datos

En este notebook trabajaremos sobre el dataset original extraído del portal de datos abiertos de la nación:

https://datos.gob.ar/dataset/justicia-sistema-nacional-estadisticas-sobre-ejecucion-pena---sneep

El objetivo de este trabajo será el de procesar el conjunto de datos para poder utilizarlos en el análisis exploratorio de los mismos. Los entregables del trabajo serán:

1- Dataset con características similares al entregado para el primer práctico:

https://github.com/camporeale/Datos/raw/master/sneep_2017_diplodatos.zip

2- Notebook con el análisis del dataset y los pasos ejecutados para la limpieza del mismo.

3- Script de python que permita automatizar el procesamiento. Debe poder pasarsele por parámetros la versión 2002-2017 del dataset y ejecutar correctamente.

### Inicialización y carga de datos

In [0]:
# Importamos las librerías necesarias
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_columns', 100)


In [0]:
# Cargamos el CSV en un dataframe
url = 'http://datos.jus.gob.ar/dataset/6c03af36-6a1d-4306-b2a8-dd39ad73afb3/resource/af0a64da-6d06-45cf-a86c-de00f09221d8/download/sneep-2017.csv'
data_raw = pd.read_csv(url,index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Análisis

In [0]:
# Obtenemos la cantidad de filas y columnas
print("Cantidad de registros: ", data_raw.shape[0])
print("Cantidad de columnas: ", data_raw.shape[1])

Cantidad de registros:  85283
Cantidad de columnas:  86


In [0]:
# Listamos las columnas y sus tipos de datos
data_raw.dtypes

provincia_id                                            int64
provincia_descripcion                                  object
establecimiento_id                                      int64
establecimiento_descripcion                            object
edad                                                  float64
genero_id                                               int64
genero_descripcion                                     object
nacionalidad_id                                         int64
nacionalidad_descripcion                               object
estado_civil_id                                       float64
estado_civil_descripcion                               object
nivel_instruccion_id                                  float64
nivel_instruccion_descripcion                          object
ultima_situacion_laboral_id                           float64
ultima_situacion_laboral_descripcion                   object
capacitacion_laboral_al_ingresar_id                   float64
capacita

In [0]:
# Analizamos 2 muestras de filas, usamos transpose() para visualizar los datos con mayor comodidad
data_raw.sample(2).transpose()

anio_censo,2017,2017
provincia_id,1,1
provincia_descripcion,Buenos Aires,Buenos Aires
establecimiento_id,219,195
establecimiento_descripcion,UNIDAD 9 - LA PLATA,UNIDAD 1 - LISANDRO OLMOS
edad,27,36
genero_id,1,1
genero_descripcion,Masculino,Masculino
nacionalidad_id,1,1
nacionalidad_descripcion,Argentina,Argentina
estado_civil_id,6,1


### Lineamientos básicos del preprocesamiento 



* Parseo de fechas. Los campos "fecha_detencion" y "fecha_condena" deben ser transformados a tipos de datos date, en este proceso puede que se encuentren errores, analizar los mismos y decidir si conservar o descartar registros con problemas.

* Análisis y Selección de Features. El dataset tiene campos duplicados que expresan la misma información: campos con sufijo "_id" y "_descripcion". En general estos campos son de tipo numérico y string respectivamente, pero hay algunas excepciones. Identificarlas y editar el dataframe para que quede consistente, conservar solo los campos con las descripciones.
 
* Análisis de consistencia de los datos 1: Analizar los valores del campo "mujer_tiene_hijos_intramuro", identificar el problema que afecta a los datos, y corregir el dataset acorde a esto. 

* Análisis de consistencia de los datos 2: Analizar los campos "fecha_detencion" y "fecha_condena"

* Generación de nuevas Features. Generar un nuevo campo que contenga información acerca del tiempo en días que toma la resolución de la situación procesal de cada persona (diferencia entre fecha de encarcelamiento y de condena)

* Análisis e imputación de valores nulos. Existen muchos registros con valores nulos en múltiples columnas. Analizar las posibles soluciones. Tener en cuenta que dependiendo el tipo de trabajo que querramos hacer con los datos en un futuro, es probable que necesitemos realizar tareas como: eliminar registros, eliminar columnas, cargar datos con valores aleatorios, cargar datos con valores más frecuentes, etc. 

* Bonus:  El dataset contiene información respecto a individuos particulares, y si bien no hay nombres o identificadores únicos como el número de documento, es muy posible que se pueda inferir la identidad de muchas personas usando otros conjuntos de datos públicos. Teniendo en cuenta que uno de los campos describe si la persona recibió atención médica por distintos tipos de enfermedades (HIV, Chagas, Hepatitis, etc.), es importante pensar en qué tipo de acciones podrían tomarse en la elaboración del reporte para asegurar la anonimidad de los datos, o de no ser posible, la eliminación de aquellos atributos que podrían ser considerados información sensible y/o privada. Hacer un análisis corto respecto a este tema y exponer sus opiniones.